In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
#Carregando o dataset
df = pd.read_excel("../dataset/dry_bean_dataset.xls")

In [3]:
#Dividindo a base em treino e teste
X = df.drop("Class", axis=1)
y = pd.get_dummies(df["Class"])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=123)

#Normalizando os dados
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)